In [79]:
import pandas as pd
import os
import sys
import MeCab

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# データフレームを表示するときカラムを省略しない
pd.set_option('display.max_columns', None)
# 浮動小数点を表示するときは小数点以下 2 桁で揃える
pd.options.display.float_format = '{:0.2f}'.format

In [93]:
def word_count(corpus):
    # 単語の数をカウントする
    count_vectorizer = CountVectorizer()
    X_count = count_vectorizer.fit_transform(corpus)

    # 見やすさのために表示するときは pandas のデータフレームにする
    df = pd.DataFrame(data=X_count.toarray(),
                      columns=count_vectorizer.get_feature_names())
    return X_count.toarray()

In [81]:
def tf_idf(corpus):
    # scikit-learn の TF-IDF 実装
    tfidf_vectorizer = TfidfVectorizer()
    X_tfidf = tfidf_vectorizer.fit_transform(corpus)

    # TF-IDF を表示する
    df = pd.DataFrame(data=X_tfidf.toarray(),
                      columns=tfidf_vectorizer.get_feature_names())
    return df

In [82]:
def word_tf_idf(review_df,tfidf_df,word):
    sorted=tfidf_df.loc[:,[word]].sort_values(word,ascending=False)
    for i in list(sorted.index):
        if tfidf_df.at[i,word] == 0:
            break
        print(str(review_df.at[i,'title'])+' : '+str(tfidf_df.at[i,word]))

In [83]:
review_df = pd.read_csv("1005data.csv")
review_df = review_df.rename(columns={'Unnamed: 0':'index', 'Title':'title', 'Rating Score':'score', 'Reviews':'review'}) 
review_df

,index,title,score,review
0,0,アフタースクール,4.09,['レビュアー試写会に何気なく応募したら何気なく当選。ありがとう！と感謝しながら４月２３日、...
1,1,シュガー・ラッシュ,4.17,['いきなり、評価ですが、今作は、苦汁の結論ですが、敢えて☆４つ。他のレビュアーとは、異なる...
2,2,コルボッコロ,2.14,['コルボッコロ、監督コメント付きで観て来た。コルボッコロ、言いづらいわ！鈴とコルボッコロの...
3,3,ロサンゼルス女子刑務所,2.24,['\u3000…あらすじは解説のとおりと書こうと思ったら空欄で、レビュー数は未だ0件だ。\...
4,4,雪の華,2.97,['この作品、正直に言ってしまうと、ありえないことだらけなんですよ。予告編やオフィシャルサイ...
...,...,...,...,...
475,475,THE 有頂天ホテル,3.36,['\u3000とても良く出来てはいるのですが、大爆笑とか深い感動とかを期待するのもどうかと...
476,476,パージ：アナーキー,3.46,"[""『パージ：アナーキー』(THE PURGE：ANARCHY／2014／米)は､前作同様ジ..."
477,477,Dear Friends ディア フレンズ,3.41,['Yoshiの原作がひどいのか､それとも脚本と監督がひどいのかはわからないが､まさしく少女...
478,478,エージェント・ウルトラ,3.07,['予告編に騙されることが多いとです（笑）最近も「人生の約束」はがっかりでしたが、今回も見事...


In [84]:
def analyze(text):
    tagger = MeCab.Tagger("-Ochasen")
    tagger.parse('')
    node = tagger.parseToNode(text)
    wakati = ''
    while node:
        word = node.surface
        wclass = node.feature.split(',')
        #mecabは以下の順でnode.featureに情報を入れる　[0]品詞,[1]品詞細分類1,[2]品詞細分類2,[3]品詞細分類3,[4]活用形,[5]活用型,[6]原形,[7]読み,[8]発音
        if (
            #wclass[0] == '記号' or
            wclass[0] == '形容詞' or
            #wclass[0] == '助詞' or
            #wclass[0] == '助動詞' or
            #wclass[0] == '接続詞' or
            #wclass[0] == '接頭詞' or
            wclass[0] == '動詞' or
            #wclass[0] == '副詞' or 
            wclass[0] == '名詞'
            #wclass[0] == '連体詞'
        ):
            if wclass[6] != '*':
                wakati += wclass[6] + ' '
        node = node.next
    return wakati


In [85]:
def make_data(pd):
    training = []
    for index in pd.index:
        sent = analyze(pd.at[index,'review'])
        training.append(sent)
    
    return training

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [87]:
tfidf_df = tf_idf(make_data(review_df))

In [95]:
word_tf_idf(review_df,tfidf_df,"冒険")

グーニーズ : 0.36672092613576024
ペット : 0.09494893602708794
ハン・ソロ／スター・ウォーズ・ストーリー : 0.04831776898207306
くるみ割り人形と秘密の王国 : 0.043032594683746844
エージェント・ウルトラ : 0.037874640347930665
テラビシアにかける橋 : 0.02922029903019794
シュガー・ラッシュ : 0.028288286011858104
ANEMONE／交響詩篇エウレカセブン　ハイエボリューション : 0.027810925790913574
花とアリス殺人事件 : 0.027319531459452776
ディセント : 0.02496754417233589
トータル・リコール : 0.021602979607569992
バック・トゥ・ザ・フューチャー : 0.020863982809116166
ナインスゲート : 0.019873157558793826
ボス・ベイビー : 0.01965856977796489
ボス・ベイビー : 0.01965856977796489
映画クレヨンしんちゃん　ガチンコ！逆襲のロボとーちゃん : 0.019131278954736163
デビルマン : 0.01855303318616348
海賊じいちゃんの贈りもの : 0.015544629428763314
Shall we ダンス？ : 0.014454529196663441
トータル・リコール : 0.012973971364043293
ライフ・オブ・パイ／トラと漂流した227日 : 0.012474641684316097
おおかみこどもの雨と雪 : 0.012364466237944486
蛇にピアス : 0.011955595603435341
蛇にピアス : 0.011955595603435341
蛇にピアス : 0.011955595603435341
キャスト・アウェイ : 0.0119399580196036
ホットロード : 0.011851716155916969
ブラック会社に勤めてるんだが、もう俺は限界かもしれない : 0.01159149157046841
プリデスティネーション : 0.01133455